Travail developpe par le binome :
**Bruno OLIVEIRA, Samuel GHEZI**

Sur orientation de le Professeur **Martin GHIENNE**.

**Introduction**

L’exploitation d’un avion s’accompagne de conditions de vol très variables et difficiles à prédire, ce qui rend complexe l’estimation précise des chargements structuraux rencontrés en situation réelle. Bien que des informations telles que les déformations et les contraintes soient essentielles pour optimiser la maintenance et améliorer les modèles de dimensionnement, ces grandeurs ne sont généralement pas mesurées directement sur les aéronefs commerciaux. L’installation de capteurs dédiés entraînerait en effet une augmentation significative des coûts, de la masse, de la complexité d’intégration et des exigences de certification.

Le Mini-Challenge propose ainsi de développer un capteur virtuel basé sur des méthodes d’apprentissage automatique, capable d’estimer l’état de contrainte structurelle en différents points de l’avion à partir des seuls paramètres déjà enregistrés par l’instrumentation de bord. L’objectif est de prédire des grandeurs non mesurées physiquement, mais inférées à partir de variables de vol telles que l’attitude, les vitesses, les accélérations, les ordres de commande et les conditions de vent.

Pour cela, un ensemble de données réelles provenant de 44 vols d’essai est mis à disposition. Ce jeu de données comprend :

39 paramètres issus de l’instrumentation de bord, représentant l’état de vol, les efforts aérodynamiques et les actions de contrôle ;

15 jauges d’extensométrie (en micro-déformations, με) positionnées en différents points structuraux de l’appareil, permettant de mesurer directement les contraintes locales.

En résumé, ce projet vise à démontrer la capacité d’un modèle d’apprentissage supervisé à reproduire les contraintes structurelles réelles à partir de données opérationnelles courantes, ouvrant la voie à des stratégies de maintenance plus prédictives, moins coûteuses et mieux informées, dans la continuité des travaux précédents sur les capteurs virtuels pour le suivi de santé structurale.

Pour cette projet, on va se baser dans la teorie de Machine Learning, en se basent dans l'image ci dessous :

In [ ]:
# from IPython.display import Image
# (Image(filename="Workflow_DL.png"))

**Management des données**

Le liste de variable avec les description et sont format qui existe dans les données collectées pendant les vols sont en dessous :

In [ ]:
# (Image(filename="table.png"))


In [15]:
# Importing Lybraries
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm import tqdm



**Management des données** - Acquisition des données

In [ ]:
# Creating the dataframes data[Tol_1], ..., data[Tol_48]
# data = {}
# for file in glob.glob('./Data_AirLoadBench/TOL_*.csv'):
#     base = os.path.basename(file)
#     name = base.replace('.csv', '')
#     data[name]  = pd.read_csv(file)

In [ ]:
# Displaying the colums existing in each dataframe
data['TOL_1'].columns

Displayng the dataframes

In [ ]:
data['TOL_1']

In [ ]:
data['TOL_1'].columns

In [ ]:
data['TOL_2']

In [ ]:
data['TOL_3']

In [ ]:
data['TOL_4']

In [ ]:
data['TOL_5']

In [ ]:
data['TOL_6']

In [ ]:
data['TOL_7']

In [ ]:
data['TOL_8']

In [ ]:
data['TOL_9']

In [ ]:
data['TOL_10']

In [ ]:
data['TOL_11']

In [ ]:
data['TOL_12']

In [ ]:
data['TOL_13']

In [ ]:
data['TOL_14']

In [ ]:
data['TOL_15']

In [ ]:
data['TOL_17']

In [ ]:
data['TOL_18']

In [ ]:
data['TOL_19']

In [ ]:
data['TOL_20']

In [ ]:
data['TOL_21']

In [ ]:
data['TOL_22']

In [ ]:
data['TOL_23']

In [ ]:
data['TOL_24']

In [ ]:
data['TOL_25']

In [ ]:
data['TOL_26']

In [ ]:
data['TOL_27']

In [ ]:
data['TOL_28']

In [ ]:
data['TOL_29']

In [ ]:
data['TOL_30']

In [ ]:
data['TOL_31']

In [ ]:
data['TOL_33']

In [ ]:
data['TOL_34']

In [ ]:
data['TOL_35']

In [ ]:
data['TOL_36']

In [ ]:
data['TOL_37']

In [ ]:
data['TOL_38']

In [ ]:
data['TOL_39']

In [ ]:
data['TOL_40']

In [ ]:
data['TOL_41']

In [ ]:
data['TOL_42']

In [ ]:
data['TOL_43']

In [ ]:
data['TOL_44']

In [ ]:
data['TOL_46']

In [ ]:
data['TOL_48']

**Management des données** - Exploration des Données

In [ ]:
# # ============================================================
# # 1) Sélection du DataFrame principal
# # ============================================================
# df = data['TOL_1']   # Votre DataFrame d'origine


# # ============================================================
# # 2) Listes des colonnes (Y = variables de vol, X = strains)
# # ============================================================
# y_cols = [
#     'Relative_Time', 'Nz', 'Nx', 'Roll_Angle', 'Pitch_Angle', 'True_AOA',
#     'True_Sideslip', 'FPA', 'True_Heading', 'CAS', 'TAS', 'Mach', 'SAT',
#     'Baro_Alt', 'Roll_Rate', 'Pitch_Rate', 'Heading_Rate', 'Fuel_Qty1',
#     'Fuel_Qty2', 'L_Eng_Start', 'R_Eng_Start', 'L_Throttle_Pos',
#     'R_Throttle_Pos', 'L_Eng_N1', 'R_Eng_N1', 'L_Eng_N2', 'R_Eng_N2',
#     'L_Gear_Down', 'R_Gear_Down', 'N_Gear_Down', 'L_Flaperon_Pos',
#     'R_Flaperon_Pos', 'L_LEF_Pos', 'R_LEF_Pos', 'L_Rudder_Pos',
#     'L_Stab_Pos', 'R_Stab_Pos', 'Stick_Pitch', 'Stick_Roll', 'Pedal_Pos'
# ]

# x_cols = [
#     'Strain1', 'Strain2', 'Strain3', 'Strain4', 'Strain5', 'Strain6',
#     'Strain7', 'Strain8', 'Strain9', 'Strain10', 'Strain11', 'Strain12',
#     'Strain13', 'Strain14', 'Strain15'
# ]


# # ============================================================
# # 3) Conversion des colonnes en valeurs numériques
# #    (évite des erreurs si certaines colonnes contiennent du texte)
# # ============================================================
# df_numeric = df.copy()

# for col in y_cols + x_cols:
#     if col in df_numeric.columns:  # sécurité si une colonne manque
#         df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')


# # ============================================================
# # 4) Génération des graphiques de dispersion (scatter)
# #    Pour chaque strain, une figure contenant plusieurs sous-graphiques
# #    Axe X = strain, Axe Y = variable de vol
# # ============================================================
# for strain in x_cols:

#     if strain not in df_numeric.columns:
#         print(f"La colonne {strain} n'est pas présente dans le DataFrame, ignorée.")
#         continue

#     # grille 7x6 de sous-graphiques (42 cases, nous avons 41 variables Y)
#     fig, axes = plt.subplots(
#         nrows=7, 
#         ncols=6, 
#         figsize=(18, 20),
#         sharex=False, 
#         sharey=False
#     )
#     axes = axes.ravel()  # transforme la grille en liste linéaire

#     # Pour chaque variable Y, créer un scatter plot
#     for i, y in enumerate(y_cols):

#         if y not in df_numeric.columns:
#             print(f"La colonne {y} n'est pas présente dans le DataFrame, ignorée.")
#             continue

#         ax = axes[i]

#         ax.scatter(
#             df_numeric[strain],
#             df_numeric[y],
#             s=5,
#             alpha=0.5
#         )
#         ax.set_xlabel(strain, fontsize=8)
#         ax.set_ylabel(y, fontsize=8)
#         ax.tick_params(labelsize=8)

#     # Suppression des sous-graphiques inutilisés
#     for j in range(len(y_cols), len(axes)):
#         fig.delaxes(axes[j])

#     # Titre global de la figure
#     fig.suptitle(
#         f'Relation entre {strain} et les variables de vol (Données : TOL_1)',
#         fontsize=16
#     )

#     plt.tight_layout()
#     plt.savefig(f"scatter_{strain}.png", dpi=300)
#     plt.show()


# # ============================================================
# # 5) Heatmap des corrélations (optionnel mais très utile)
# #    Visualisation de la corrélation entre toutes les Y et toutes les Strains
# # ============================================================
# cols_present = [c for c in (y_cols + x_cols) if c in df_numeric.columns]

# # matrice de corrélation
# corr = df_numeric[cols_present].corr()

# # sous-matrice : lignes = Y, colonnes = Strains
# corr_sub = corr.loc[
#     [c for c in y_cols if c in corr.index],
#     [c for c in x_cols if c in corr.columns]
# ]

# plt.figure(figsize=(12, 10))
# sns.heatmap(
#     corr_sub,
#     annot=True,
#     fmt=".2f",
#     cmap='coolwarm',
#     center=0,
#     annot_kws={"size": 8} 

# )
# plt.xlabel('Strains')
# plt.ylabel('Variables de vol')
# plt.title('Corrélations entre les variables de vol et les strains (TOL_1)')
# plt.savefig("heatmap_correlations_TOL1.png", dpi=300)
# plt.tight_layout()
# plt.show()



Ploting grqphs for correlation

In [ ]:
base_path = "./"

for i in range(1, 16):
    filename = f"{base_path}scatter_Strain{i}.png"
    
    img = mpimg.imread(filename)
    plt.figure(figsize=(20, 12))   # <<< ALTERADO PARA FICAR BEM GRANDE
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"scatter_Strain{i}.png", fontsize=20)
    plt.tight_layout()
    plt.show()

In [ ]:
img = mpimg.imread("heatmap_correlations_TOL1.png")

plt.figure(figsize=(12, 10))
plt.imshow(img)
plt.axis("off")
plt.title("Heatmap des corrélations (TOL_1)")
plt.show()

**Management des données** - Préparation des Données

In [ ]:
# dfs = []  # lista para guardar todos os dataframes

# for name, df in data.items():
#     df_temp = df.copy()
#     df_temp["Tol_ID"] = name     # adiciona coluna identificando o TOL
#     dfs.append(df_temp)

# # junta tudo em um dataframe só
# df = pd.concat(dfs, ignore_index=True)

# df.to_csv('df.csv', index=False)

In [ ]:
# df = pd.read_csv('df.csv')

In [ ]:
# df = df.drop(columns='Relative_Time')

In [ ]:
# # Targets = deformações (saídas do modelo)
# target_cols = [f"Strain{i}" for i in range(1, 16)]

# # Features = todas as colunas menos Strains e Flight_ID
# feature_cols = [c for c in df.columns if c not in target_cols + ["Tol_ID"]]

# print("Features:", feature_cols)
# print("Targets :", target_cols)

In [ ]:
# # Remove linhas com NaN nas colunas importantes
# df_clean = df.dropna(subset=feature_cols + target_cols)

# X = df_clean[feature_cols].values      # entradas
# y = df_clean[target_cols].values      # saídas (strains)
# Tol_ids = df_clean["Tol_ID"].values  # de qual TOL é cada linha

In [ ]:
# scaler_X = StandardScaler()
# scaler_y = StandardScaler()

# X_scaled = scaler_X.fit_transform(X)
# y_scaled = scaler_y.fit_transform(y)

In [ ]:
# X_train, X_temp, y_train, y_temp, ids_train, ids_temp = train_test_split(
#     X_scaled, y_scaled, Tol_ids,
#     test_size=0.3, random_state=42, stratify=Tol_ids
# )

# X_val, X_test, y_val, y_test, ids_val, ids_test = train_test_split(
#     X_temp, y_temp, ids_temp,
#     test_size=0.5, random_state=42, stratify=ids_temp
# )

# print(X_train.shape, X_val.shape, X_test.shape)

In [ ]:
# class StrainDataset(Dataset):
#     def __init__(self, X, y):
#         self.X = torch.tensor(X, dtype=torch.float32)
#         self.y = torch.tensor(y, dtype=torch.float32)

#     def __len__(self):
#         return len(self.X)

#     def __getitem__(self, idx):
#         return self.X[idx], self.y[idx]

# train_ds = StrainDataset(X_train, y_train)
# val_ds   = StrainDataset(X_val,   y_val)
# test_ds  = StrainDataset(X_test,  y_test)

# train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
# val_loader   = DataLoader(val_ds,   batch_size=256, shuffle=False)
# test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False)

In [ ]:
# n_features = X_train.shape[1]
# n_outputs  = y_train.shape[1]   # 15 strains

# class StrainMLP(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(n_features, 128),
#             nn.ReLU(),
#             nn.Linear(128, 256),
#             nn.ReLU(),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Linear(128, n_outputs)
#         )

#     def forward(self, x):
#         return self.net(x)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = StrainMLP().to(device)

# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in tqdm(loader):
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)

    return running_loss / len(loader.dataset)

def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            running_loss += loss.item() * X_batch.size(0)

    return running_loss / len(loader.dataset)

n_epochs = 50

for epoch in range(1, n_epochs + 1):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    val_loss   = eval_epoch(model, val_loader, criterion)
    print(f"Epoch {epoch:03d} | Train loss: {train_loss:.6f} | Val loss: {val_loss:.6f}")

100%|██████████| 19303/19303 [01:45<00:00, 182.56it/s]


Epoch 001 | Train loss: 0.042444 | Val loss: 0.043014


100%|██████████| 19303/19303 [01:21<00:00, 236.15it/s]


Epoch 002 | Train loss: 0.041912 | Val loss: 0.042087


100%|██████████| 19303/19303 [01:21<00:00, 236.02it/s]


...

Epoch 048 | Train loss: 0.034695 | Val loss: 0.035185


100%|██████████| 19303/19303 [01:28<00:00, 218.20it/s]


Epoch 049 | Train loss: 0.034647 | Val loss: 0.034286


100%|██████████| 19303/19303 [01:28<00:00, 219.12it/s]


Epoch 050 | Train loss: 0.034571 | Val loss: 0.034464


In [ ]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# model.eval()
# with torch.no_grad():
#     X_test_t = torch.tensor(X_test, dtype=torch.float32).to(device)
#     y_pred_scaled = model(X_test_t).cpu().numpy()

# # voltar para escala original das strains
# y_test_real = scaler_y.inverse_transform(y_test)
# y_pred_real = scaler_y.inverse_transform(y_pred_scaled)

# # métricas globais
# mae_global  = mean_absolute_error(y_test_real, y_pred_real)
# rmse_global = mean_squared_error(y_test_real, y_pred_real, squared=False)
# r2_global   = r2_score(y_test_real, y_pred_real)

# print("Métricas globais no conjunto de teste:")
# print(f"MAE  : {mae_global:.4f}")
# print(f"RMSE : {rmse_global:.4f}")
# print(f"R²   : {r2_global:.4f}")

Métricas globais no conjunto de teste:
MAE  : 19.3445
RMSE : 29.3919
R²   : 0.9656


In [ ]:
# metrics_per_flight = {}
# unique_flights = np.unique(ids_test)

# for flight in unique_flights:
#     mask = (ids_test == flight)
#     y_true_f = y_test_real[mask]
#     y_pred_f = y_pred_real[mask]

#     mae  = mean_absolute_error(y_true_f, y_pred_f)
#     rmse = mean_squared_error(y_true_f, y_pred_f, squared=False)
#     r2   = r2_score(y_true_f, y_pred_f)

#     metrics_per_flight[flight] = {"MAE": mae, "RMSE": rmse, "R2": r2}

# for flight, m in metrics_per_flight.items():
#     print(f"{flight}: MAE={m['MAE']:.4f}, RMSE={m['RMSE']:.4f}, R²={m['R2']:.4f}")

# mae_mean  = np.mean([m["MAE"]  for m in metrics_per_flight.values()])
# rmse_mean = np.mean([m["RMSE"] for m in metrics_per_flight.values()])
# r2_mean   = np.mean([m["R2"]   for m in metrics_per_flight.values()])

# print("\nMédia das métricas sobre todos os TOLs (teste):")
# print(f"MAE médio  : {mae_mean:.4f}")
# print(f"RMSE médio : {rmse_mean:.4f}")
# print(f"R² médio   : {r2_mean:.4f}")

TOL_1: MAE=16.8403, RMSE=24.7917, R²=0.9683
TOL_10: MAE=17.3635, RMSE=25.3564, R²=0.9441
TOL_11: MAE=18.6012, RMSE=28.9841, R²=0.9358
TOL_12: MAE=21.6690, RMSE=32.5024, R²=0.9407
TOL_13: MAE=19.3074, RMSE=26.4903, R²=0.9527
TOL_14: MAE=22.5286, RMSE=34.1592, R²=0.9498
TOL_15: MAE=23.2700, RMSE=33.9786, R²=0.9539
TOL_17: MAE=18.6578, RMSE=25.8217, R²=0.9612
TOL_18: MAE=24.1571, RMSE=38.0362, R²=0.9318
TOL_19: MAE=19.4598, RMSE=30.1341, R²=0.9404
TOL_2: MAE=15.1694, RMSE=22.4689, R²=0.9529
TOL_20: MAE=23.2818, RMSE=34.4150, R²=0.9358
TOL_21: MAE=15.8223, RMSE=24.0751, R²=0.9677
TOL_22: MAE=14.1698, RMSE=19.3797, R²=0.9104
TOL_23: MAE=17.7110, RMSE=27.0514, R²=0.9313
TOL_24: MAE=16.7271, RMSE=24.4269, R²=0.9463
TOL_25: MAE=17.7578, RMSE=25.4399, R²=0.9655
TOL_26: MAE=20.9871, RMSE=32.9522, R²=0.9608
TOL_27: MAE=20.2276, RMSE=28.5018, R²=0.9088
TOL_28: MAE=21.1644, RMSE=31.5361, R²=0.9349
TOL_29: MAE=18.5927, RMSE=28.3030, R²=0.9484
TOL_3: MAE=15.0125, RMSE=21.0291, R²=0.9662
TOL_30: MAE=1

In [ ]:
# df_flights = pd.DataFrame.from_dict(metrics_per_flight, orient='index')

# # adicionar as métricas globais
# df_global = pd.DataFrame({
#     "MAE":  [mae_global],
#     "RMSE": [rmse_global],
#     "R2":   [r2_global]
# }, index=["GLOBAL"])

# # concatenar tudo
# df_results = pd.concat([df_flights, df_global])

# df_results

,MAE,RMSE,R2
TOL_1,16.840289,24.791730,0.968317
TOL_10,17.363486,25.356402,0.944072
TOL_11,18.601200,28.984143,0.935757
TOL_12,21.668951,32.502419,0.940683
TOL_13,19.307354,26.490346,0.952732
TOL_14,22.528594,34.159154,0.949833
TOL_15,23.269967,33.978556,0.953948
TOL_17,18.657814,25.821662,0.961208
TOL_18,24.157051,38.036220,0.931761
TOL_19,19.459846,30.134092,0.940399


In [22]:
df_results.to_csv('df_results_all_columns.csv')

In [27]:
df_results = pd.read_csv('df_results_all_columns.csv')
df_results


,Unnamed: 0,MAE,RMSE,R2
0,TOL_1,16.840289,24.791730,0.968317
1,TOL_10,17.363486,25.356402,0.944072
2,TOL_11,18.601200,28.984143,0.935757
3,TOL_12,21.668951,32.502419,0.940683
4,TOL_13,19.307354,26.490346,0.952732
5,TOL_14,22.528594,34.159154,0.949833
6,TOL_15,23.269967,33.978556,0.953948
7,TOL_17,18.657814,25.821662,0.961208
8,TOL_18,24.157051,38.036220,0.931761
9,TOL_19,19.459846,30.134092,0.940399
